<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import  make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error,mean_squared_error,make_scorer, f1_score
from sklearn.dummy import DummyRegressor

In [2]:
data=pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
data.duplicated().sum()

0

In [6]:
data.isnull().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

Не выявлено дубликатов или пропусков, можем работать с дата фреймом дальше

In [7]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Значительная разница, признак несбалансированный 

In [8]:
lemmatizer = nltk.WordNetLemmatizer()

def lemmatize_text(text):
    text = text.lower()
    lemm_text = []
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    for word, tag in tagged:
        pos = get_wordnet_pos(tag)
        if pos:
            lemm_text.append(lemmatizer.lemmatize(word, pos))
        else:
            lemm_text.append(lemmatizer.lemmatize(word))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', " ".join(lemm_text)) 
    return " ".join(cleared_text.split())

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [9]:
def clear_text(text):
    re_list = re.sub(r"[^a-zA-Z']", ' ', text)
    re_list = re_list.split()
    re_list = " ".join(re_list)
    return re_list

In [10]:
data['clear_text'] = data['text'].apply(clear_text)
data['lemm_text'] = data['clear_text'].apply(lemmatize_text)
data = data.drop(['text'], axis=1)

In [11]:
data.head()

,Unnamed: 0,toxic,clear_text,lemm_text
0,0,0,Explanation Why the edits made under my userna...,explanation why the edits make under my userna...
1,1,0,D'aww He matches this background colour I'm se...,d aww he match this background colour i m seem...
2,2,0,Hey man I'm really not trying to edit war It's...,hey man i m really not try to edit war it s ju...
3,3,0,More I can't make any real suggestions on impr...,more i ca n t make any real suggestion on impr...
4,4,0,You sir are my hero Any chance you remember wh...,you sir be my hero any chance you remember wha...


**Вывод**

Выполнили предобработку:

    1) Не обнаружили явных дубликатов и пропусков. 
    2) Заметили, что признак toxic не сбалансирован, из-за чего будем использовать balanced при обучении модели
    3) Очистили текст от лишних символов и лемматизировали его

## Обучение

In [12]:
features=data.drop(['toxic'], axis=1)
target = data['toxic']
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words = stopwords)

corpus_train= features_train['lemm_text'].values
corpus_test= features_test ['lemm_text'].values

tf_idf_train = count_tf_idf.fit_transform(corpus_train)
tf_idf_test =count_tf_idf.transform(corpus_test)

print(tf_idf_train.shape)
print(tf_idf_test.shape)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(127433, 136640)
(31859, 136640)


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Здорово , что у нас есть выборка для тестов!</font>

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Убрали частые неинформативные слова!</font>

In [20]:
model_lr = Pipeline([('scaler', MaxAbsScaler()),('regressor', LogisticRegression(solver='liblinear',class_weight='balanced',random_state = 12345))])  
parameters_lr = {}
grid_search_lr = GridSearchCV(model_lr, param_grid=parameters_lr,  scoring='f1',cv=5)
grid_search_lr.fit(tf_idf_train,target_train)
print("Лучшие гиперпараметры для LogisticRegression: ", grid_search_lr.best_params_)
print("Лучшее значение метрики для LogisticRegression: ", (grid_search_lr.best_score_))

Лучшие гиперпараметры для LogisticRegression:  {}
Лучшее значение метрики для LogisticRegression:  0.7490588942735523


In [14]:
model_dtс = Pipeline([('scaler', MaxAbsScaler()),('regressor', DecisionTreeClassifier(random_state=12345,class_weight='balanced'))])
parameters_dtс = {'regressor__max_depth':range(1,10)}
grid_search_dtс = GridSearchCV(model_dtс, param_grid=parameters_dtс,  scoring='f1',cv=5)
grid_search_dtс.fit(tf_idf_train,target_train)
print("Лучшие гиперпараметры для DecisionTreeClassifier: ", grid_search_dtс.best_params_)
print("Лучшее значение метрики для DecisionTreeClassifier: ", (grid_search_dtс.best_score_))

Лучшие гиперпараметры для DecisionTreeClassifier:  {'regressor__max_depth': 9}
Лучшее значение метрики для DecisionTreeClassifier:  0.5411789329027114


На тренировочной выборке лучше справилась модель LogisticRegression, перейдем к её тестированию 

## Выводы

In [21]:
model_lr=LogisticRegression(solver='liblinear', class_weight='balanced',random_state = 12345)
model_lr.fit(tf_idf_train,target_train)
prediction=model_lr.predict(tf_idf_test)
print("f1 =", f1_score(target_test,prediction))

f1 = 0.7421080791867309


Модель имеет значение 0.75 что является нижней границей условий, значит модель подходит

# Вывод

Выполнили предобработку:

    1) Не обнаружили явных дубликатов и пропусков. 
    2) Заметили, что признак toxic не сбалансирован, из-за чего будем использовать balanced при обучении модели
    3) Очистили текст от лишних символов и лемматизировали его

Далее обучили модели LogisticRegression, DecisionTreeClassifier. На тестовой выборке лучшее значение показала логистическая регрессия, поэтому проверили ее на тестовой выборке, где она показала результат в 0.75, что удовлетворяет нашим условиям

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны